<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0111Fine_Tune_Speech_Recognition_Model_with_Transformers_using_CTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning Speech Model with 🤗 Transformers**

このノートでは，自動音声認識のための多言語の事前学習済み音声モデルを微調整する方法を紹介します。
<!-- This notebook shows how to fine-tune multi-lingual pretrained speech models for Automatic Speech Recognition. -->

このノートブックは [TIMITデータセット](https://huggingface.co/datasets/timit) 上で，[モデルハブ Model Hub](https://huggingface.co/models?pipeline_tag=automatic-speech-recognition&sort=downloads) から任意の音声モデルのチェックポイントを実行するように構築されています。
そのモデルに CTC (Connectionist Temporal Classification) ヘッドを搭載したバージョンがある場合に限ります。
モデルや使用している GPU によっては，メモリ不足のエラーを回避するために，バッチサイズを調整する必要があるかもしれません。
この 2 つのパラメータを設定すれば，あとはノートブックがスムーズに動作するはずです。
<!-- This notebook is built to run on the [TIMIT dataset](https://huggingface.co/datasets/timit) with any speech model checkpoint from the [Model Hub](https://huggingface.co/models?pipeline_tag=automatic-speech-recognition&sort=downloads) as long as that model has a version with a Connectionist Temporal Classification (CTC) head. 
Depending on the model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. 
Set those two parameters, then the rest of the notebook should run smoothly: -->

In [1]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

多言語訓練済音声モデルがどのように機能するのか，より詳細な説明については [🤗ブログ](https://huggingface.co/blog/fine-tune-wav2vec2-english) をご覧ください。
<!-- For a more in-detail explanation of how multi-lingual pretrained speech models function, please take a look at the [🤗 Blog](https://huggingface.co/blog/fine-tune-wav2vec2-english). -->

始める前に，マスターから `datasets` と `transformers` の両方をインストールしましょう。 
また，音声ファイルを読み込むためには `librosa` パッケージが必要で，微調整されたモデルを [word error rate (WER)](https://huggingface.co/metrics/wer) メトリック ${}^1$ を用いて評価するためには `jiwer` が必要です。
<!-- Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$. -->

In [2]:
%%capture
!pip install datasets==1.14
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

次に，訓練中に，訓練チェックポイントを [🤗 Hub](https://huggingface.co/) に直接アップロードすることを強くお勧めします。
[🤗 Hub](https://huggingface.co/) にはバージョン管理が統合されているので，訓練中にモデルのチェックポイントが失われていないことを確認することができます。

これを行うには，Hugging Face ウェブサイトから認証トークンを保存する必要があります (まだ登録していない場合は [こちらから](https://huggingface.co/join) で登録してください！)。
<!-- Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. 
The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training. 

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) -->

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

そして，モデルのチェックポイントをアップロードするために，Git-LFS をインストールする必要があります。
<!-- Then you need to install Git-LFS to upload your model checkpoints: -->

In [4]:
%%capture
!apt install git-lfs




---

${}^1$ Timitは通常，音素誤り率 (PER) を用いて評価されますが ASR で最も一般的な指標は圧倒的に単語誤り率 (WER) です。
このノートをできるだけ一般的なものにするために，WER を使ってモデルを評価することにしました。
<!-- ${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). 
To keep this notebook as general as possible we decided to evaluate the model using WER. -->

## データの準備，トークン化器，特徴抽出器
<!-- ## Prepare Data, Tokenizer, Feature Extractor -->

ASR モデルは音声をテキストに変換するため，音声信号をモデルの入力フォーマット (例: 特徴ベクトル) に処理する特徴抽出器と，モデルの出力フォーマットをテキストに処理するトークン化器の両方が必要となります。
<!-- ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. -->

このように🤗 Transformersでは，音声認識モデルにはトークン化器と特徴抽出器の両方が付属しています。
<!-- In 🤗 Transformers, speech recognition models are thus accompanied by both a tokenizer, and a feature extractor. -->

まずは，モデルの予測値を復号化する役割を持つトークン化器を作ってみましょう。
<!-- Let's start by creating the tokenizer responsible for decoding the model's predictions. -->

### Wav2Vec2CTCTokenizer の作成
<!-- ### Create Wav2Vec2CTCTokenizer -->

まず [TIMITデータセット](https://huggingface.co/datasets/timit) を読み込み，その構造を見てみましょう。
<!-- Let's start by loading the [TIMIT dataset](https://huggingface.co/datasets/timit) and taking a look at its structure.-->

もし，別の [音声データセット](https://huggingface.co/datasets?task_categories=task_categories:speech-processing&sort=downloads) でモデルを微調整したい場合は，この部分を自由にアレンジしてください。
<!-- If you wish to fine-tune the model on a different [speech dataset](https://huggingface.co/datasets?task_categories=task_categories:speech-processing&sort=downloads) feel free to adapt this part. -->

In [ ]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

In [ ]:
timit

多くの ASR データセットでは，各音声  `'audio’` やファイル `'file’` に対して，ターゲットとなるテキスト `'text’` のみを提供しています。
Timit は実際には各オーディオファイルについて `'phonetic_detail’` などのより多くの情報を提供しており，多くの研究者が Timit を扱う際に，音声認識ではなく音素分類でモデルを評価することを選択するのはそのためです。
しかし，私たちはノートブックをできるだけ一般的なものにしたいので，微調整のために転写されたテキストのみを考慮することになります。
<!-- Many ASR datasets only provide the target text, `'text'` for each audio `'audio'` and file `'file'`. 
Timit actually provides much more information about each audio file, such as the `'phonetic_detail'`, etc., which is why many researchers choose to evaluate their models on phoneme classification instead of speech recognition when working with Timit. 
However, we want to keep the notebook as general as possible, so that we will only consider the transcribed text for fine-tuning.
-->


In [7]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

データセットのランダムなサンプルを表示する短い関数を書き，それを 2, 3 回実行して transcription (転写) の感触を確かめてみましょう。
<!-- Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions. -->

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)

よしっ！と思いました。トランスクリプションはとてもきれいで，言語は対話よりも書かれたテキストに対応しているように見えます。
これは [Timit](https://huggingface.co/datasets/timit_asr) が読み上げ音声コーパスであることを考慮すると納得できます。
<!-- Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. 
This makes sense taking into account that [Timit](https://huggingface.co/datasets/timit_asr) is a read speech corpus. -->

トランスクリプションには、`,.?!;:`のような特殊文字が含まれていることがわかります。
言語モデルがない場合、このような特殊文字は特徴的な音の単位に対応していないため、スピーチチャンクを分類するのは非常に困難です。
例えば、文字`"s"`は多かれ少なかれ明確な音を持っていますが、特殊文字`"."`はそうではありません。
また、音声信号の意味を理解するためには、通常、特殊文字を文字起こしに含める必要はありません。
<!-- We can see that the transcriptions contain some special characters, such as `,.?!;:`. 
Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. 
E.g., the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.-->

さらに、テキストを小文字のみに正規化し、最後に単語区切りのトークンを追加します。
<!--In addition, we normalize the text to only have lower case letters and append a word separator token at the end. -->

In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [11]:
timit = timit.map(remove_special_characters)

  0%|          | 0/4620 [00:00<?, ?ex/s]

  0%|          | 0/1680 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

いいですねー。
これは良い感じですね。
トランスクリプションからほとんどの特殊文字を取り除き，小文字のみに正規化しました。
<!-- Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.-->

CTC では，音声の塊を文字に分類するのが一般的なので，ここでも同じように分類します。
訓練データとテストデータの異なる文字をすべて抽出し，この文字のセットから語彙を構築しましょう。
<!-- In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters. -->

すべての書き起こし文字を1つの長い書き起こし文字に連結し，その文字列を文字の集合に変換するマッピング関数を書きます。
`map(…)` 関数に引数 `batched=True` を渡して，マッピング関数がすべての転写文字に一度にアクセスできるようにすることが重要です。
<!--We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once. -->

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = timit.map(
  extract_all_chars,
  batched=True,
  batch_size=-1,
  keep_in_memory=True, 
  remove_columns=timit.column_names["train"]
)

ここで訓練データセットとテストデータセットに含まれるすべての文字の組合わせを作り，その結果得られたリストを列挙型の辞書に変換します。
<!-- Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary. -->

In [15]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

データセットにはすべてのアルファベットが含まれており (これは驚くべきことではありません)，また，特殊文字 `" "` と `'` も抽出されています。これらの特殊文字を除外していないことに注意してください。
<!-- Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because: -->

- モデルは，単語がいつ終わるかを予測することを学習しなければなりません。
さもなければ，モデルの予測は常に文字の連続となり，単語を互いに分離することができなくなってしまいます。
- 英語では，単語を区別するために `'` という文字を残しておく必要があります。
例えば，`"it's"` と `"its"` は全く異なる意味を持ちます。

<!--
- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings. -->


また，`”"` が独自のトークンクラスを持っていることを明確にするために，より目につきやすい文字 `|` を与えています。
さらに Timit の訓練セットでは遭遇しなかった文字をモデルが扱うことができるように unknown トークンも追加しました。
<!-- To make it clearer that `" "` has its own token class, we give it a more visible character `|`. 
In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set. -->

最後に CTC の "*blank token*” に相当するパディング・トークンも追加します。
「ブランクトークン」は CTC アルゴリズムの中核をなす要素です。
詳しくは「アライメント」[こちらの項](https://distill.pub/2017/ctc/) をご覧ください。
<!--
Finally, we also add a padding token that corresponds to CTC's "*blank token*". 
The "blank token" is a core component of the CTC algorithm. 
For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/). -->

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

これで 30 個のトークンからなる語彙が完成しました。
つまり，訓練済みスピーチチェックポイントの上に追加する線形層の出力次元は 30 になります。
<!-- Cool, now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained speech checkpoint will have an output dimension of 30. -->

それでは，語彙を json ファイルとして保存してみましょう。
<!-- Let's now save the vocabulary as a json file. -->

In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

最後のステップでは json ファイルを使って，作成したばかりの語彙ファイルを使ってトークン化器オブジェクトを実体化します。
正しい `tokenizer_type` はモデルのコンフィグレーションから取得することができます。
もし，コンフィグで `tokenizer_class` が定義されていれば，その定義を使用することができます。
そうでなければ `tokenizer_type` は `model_type` に対応していると仮定します。
<!-- In a final step, we use the json file to instantiate a tokenizer object with the just created vocabulary file. 
The correct `tokenizer_type` can be retrieved from the model configuration. 
If a `tokenizer_class` is defined in the config, we can use it, else we assume the `tokenizer_type` corresponds to the `model_type`. -->

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

これで `AutoTokenizer` を使って，トークン化器を実体化することができます。
さらに，トークン化器の特別なトークンを設定します。
<!-- Now we can instantiate a tokenizer using `AutoTokenizer`. 
Additionally, we set the tokenizer's special tokens. -->

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

作成したばかりのトークン化器を，このノートブックの微調整されたモデルで再利用したい場合には [🤗 Hub](https://huggingface.co/) に`tokenizer` をアップロードすることを強くお勧めします。
ここでは，ファイルをアップロードするリポジトリを `"wav2vec2-base-timit-demo-colab"` としておきます。
<!-- If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). 
Let's call the repo to which we will upload the files `"wav2vec2-base-timit-demo-colab"`: -->

In [22]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = f"{model_checkpoint_name}-demo-colab"

そして，そのトークン化器を [🤗 Hub](https://huggingface.co/)にアップロードします。
<!-- and upload the tokenizer to the [🤗 Hub](https://huggingface.co/). -->

In [ ]:
tokenizer.push_to_hub(repo_name)

これで `https://huggingface.co/<あなたのユーザ名>/wav2vec2-base-timit-demo-colab`に作成されたばかりのリポジトリを見ることができます。
<!-- Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-base-timit-demo-colab` -->

### データの前処理
<!-- ### Preprocess Data -->

ここまでは，音声信号の実際の値ではなく，書き起こしたものを見てきました。
データセットには `'text'`に加えて，`'file’` と `'audio’` というカラム名があります。
`'file’` には音声ファイルの絶対パスが入ります。
ちょっと見てみましょう。
<!-- So far, we have not looked at the actual values of the speech signal but just the transcription. 
In addition to `'text'`, our datasets include two more column names `'file'` and `'audio'`. `'file'` states the absolute path of the audio file. Let's take a look. -->

In [ ]:
timit["train"][0]["file"]

`Wav2Vec2` は 16 kHz の 1 次元配列の形式で入力を求めます。
つまり，オーディオファイルを読み込んでリサンプリングする必要があります。
<!-- `Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. 
This means that the audio file has to be loaded and resampled.-->

ありがたいことに `datasets`  は，列 `audio` を呼び出す際に，自動的にこれを行います。
では，実際に試してみましょう。
<!-- Thankfully, `datasets` does this automatically when calling the column `audio`. 
 Let try it out.  -->

In [ ]:
timit["train"][0]["audio"]

オーディオファイルが自動的に読み込まれていることがわかります。
これは `datasets == 1.13.3` で導入された新しい [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) のおかげで，呼び出し時にオンザフライでオーディオファイルをロードして再サンプリングします。
<!-- We can see that the audio file has automatically been loaded. 
This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 1.13.3`, which loads and resamples audio files on-the-fly upon calling.-->

サンプリングレートは 16 kHzに設定されており，これは`Wav2Vec2` が入力として期待するものです。
<!-- The sampling rate is set to 16kHz which is what `Wav2Vec2` expects as an input. -->

それでは，データセットの理解を深め，音声が正しく読み込まれたことを確認するために，いくつかの音声ファイルを聴いてみましょう。
<!-- Great, let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. -->

**注**: *以下のセルを何度かクリックすると，異なる音声サンプルを聞くことができます。
<!-- **Note**: *You can click the following cell a couple of times to listen to different speech samples.* -->

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

話者の話す速度やアクセントなどが変化していくのがわかります。
しかし，全体的には比較的クリアな音で録音されています。
<!-- It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus. -->

データが正しく準備されているかどうか，音声入力の形状，転写，対応するサンプリングレートを印刷して最終確認してみましょう。
<!-- Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate. -->

**注** 複数のサンプルを確認するために，以下のセルを数回クリックすることができます。
<!-- **Note**: *You can click the following cell a couple of times to verify multiple samples.* -->

In [ ]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

いいですねー。データが 1 次元配列であること，サンプリングレートが常に 16 kHzに対応していること，対象となるテキストが正規化されていることなど，すべて順調です。

次は，モデルの特徴抽出器でデータを処理します。
特徴抽出器をロードしましょう
<!-- Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Next, we should process the data with the model's feature extractor. Let's load the feature extractor -->

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

トークン化器と一緒に Wav2Vec2Processor に包み込みます。
<!-- and wrap it into a Wav2Vec2Processor together with the tokenizer. -->

In [33]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

最後に `Wav2Vec2Processor` を利用して，学習用モデルが期待する形式にデータを処理することができます。
そのために Dataset の[`map(...)`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=map#datasets.DatasetDict.map) 関数を利用しましょう。
<!-- Finally, we can leverage `Wav2Vec2Processor` to process the data to the format expected by the model for training. 
To do so let's make use of Dataset's [`map(...)`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=map#datasets.DatasetDict.map) function.-->

まず `batch["audio”]` を呼び出すだけで，オーディオデータを読み込み，再サンプリングします。
次に，読み込んだオーディオファイルから `input_values` を抽出します。今回の例では `Wav2Vec2Processor` はデータを正規化するだけです。
しかし，他の音声モデルでは，このステップに [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum) のような，より複雑な特徴抽出を含めることができます。
3 つ目は，トランスクリプションをラベル ID に符号化することです。
<!-- First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids. -->

In [34]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

データ準備機能をすべての例に適用してみましょう。
<!-- Let's apply the data preparation function to all examples. -->

In [35]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

**注**: 現在 `datasets` では，オーディオのロードと再サンプリングに [`torchaudio`](https://pytorch.org/audio/stable/index.html)  と [`librosa`](https://librosa.org/doc/latest/index.html) を使用しています。
もし，独自のコストでデータの読み込みやサンプリングを行いたい場合は，`"path”` 列を利用し `"audio”` 列は無視しても構いません。
<!-- **Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. 
If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column. -->

長い入力系列は多くのメモリを必要とします。
`Wav2Vec2` は `自己注意` に基づいているため，長い入力系列の場合，必要なメモリ量は入力の長さに対して 2 次関数的に増加します ( [この reddit 投稿](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) を参照してください)。
今回のデモでは，訓練データセットから 4 秒以上の系列をすべてフィルタリングしてみましょう。
<!-- Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). 
For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset. -->

In [ ]:
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

素晴らしい！これでトレーニングを始める準備ができました。
<!-- Awesome, now we are ready to start training! -->

## 訓練
<!-- ## Training -->

データが処理されたので，学習パイプラインの設定を開始する準備ができました。
ここでは🤗の [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) を利用しますが，そのためには基本的に以下のことが必要になります。
<!-- The data is processed so that we are ready to start setting up the training pipeline. 
We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:-->

- データコリレーターを定義する。多くの NLP モデルとは対照的に，音声モデルは通常，入力長が出力長よりもはるかに大きいです。
*例えば* Wav2Vec2 の入力長が 50000 のサンプルは，出力長が 100 以下となります。
つまり，すべての訓練サンプルは，そのバッチの中で最も長いサンプルにのみパディングされ，全体の中で最も長いサンプルにはパディングされないということです。
そのため，音声モデルの微調整には，特別なパディング・データ・コレータが必要となりますが，これを以下に定義します。

<!-- - Define a data collator. In contrast to most NLP models, speech models usually have a much larger input length than output length. 
*E.g.*, a sample of input length 50000 for Wav2Vec2 has an output length of no more than 100. 
Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. 
Therefore, fine-tuning speech models requires a special padding data collator, which we will define below -->

- 評価指標。
学習中，モデルは単語の誤り率で評価されるべきです。
そのために `compute_metrics` 関数を定義します。

<!-- - Evaluation metric. 
During training, the model should be evaluated on the word error rate. 
We should define a `compute_metrics` function accordingly -->

- 学習済みのチェックポイントを読み込む。
学習前のチェックポイントをロードして，学習用に正しく設定する必要があります。

<!--  - Load a pretrained checkpoint. 
We need to load a pretrained checkpoint and configure it correctly for training. -->

- 学習設定の定義。

<!-- - Define the training configuration. -->

モデルを微調整した後は，テストデータでモデルを正しく評価し，音声を正しく書き取ることができるようになったことを確認します。
<!-- After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech. -->

###  トレーナのセットアップ
<!-- ### Set-up Trainer -->

まず，データコリレータの定義を説明します。
データコレータのコードは [このサンプルコード](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81) からコピーしました。
<!-- Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81). -->

詳細は省きますが，一般的なデータコレータとは異なり，このデータコレータは `input_values` と `labels` を別個に扱い，それらに対して別々のパディング関数を適用します。
これは，音声の入力と出力は異なるモダリティであるため，同じパディング関数で処理すべきではないという理由からです。
一般的なデータコレータと同様，ラベル内のトークンを `-100` でパディングすることで，これらのトークンは損失を計算する際に **考慮されない**  ことになります。
<!-- Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them. 
This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss. -->

In [37]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [38]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

次に，評価指標を定義します。
前述したように ASR では WER (単語誤り率) が主流であるため，このノートでも WER を使用します。
<!-- Next, the evaluation metric is defined. As mentioned earlier, the predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well. -->

In [ ]:
wer_metric = load_metric("wer")

このモデルは確率比ベクトルの系列を返します。
ここで  $\mathbf{y}_1 = f_{theta}(x_{1}, \ldots, x_{n})[0]$ とし，$n >> m$ とすると，$\mathbf{y}_{1}=f_{\theta}(x_{1},\dots,x_{n})[0]$  となります。
<!-- The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.-->

確率比ベクトル $\mathbf{y} _{1}$ には，先ほど定義した語彙の各単語の対数が含まれているので，$\text{len}(\mathbf{y}_{i}) =$ `config.vocab_size` となります。
モデルの最も可能性の高い予測値に興味があるので，確率比の`argmax(...)`を取ります。
また，符号化されたラベルを元の文字列に戻すために `-100` を`pad_token_id` に置き換え，CTC スタイルの ${}^1$で，連続したトークンが同じトークンにグループ化されていないことを確認しながら，ids を復号化します。
<!-- A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. 
We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. 
Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$. -->

In [44]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

これで，事前訓練された `Wav2Vec2` チェックポイントをロードすることができます。
トークン化器の `pad_token_id` は，モデルの `pad_token_id` を定義するものでなければならず，CTC 音声モデルの場合には CTC の *blank token* ${}^2$ も定義します。
<!-- Now, we can load the pretrained `Wav2Vec2` checkpoint. 
The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of a CTC speech model also CTC's *blank token* ${}^2$. -->

In [ ]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

ほとんどのトランスフォーマーベースの音声モデルの最初の要素は，生の音声信号から音響的に意味のある，しかし文脈的には独立した特徴を抽出するために使用される CNN 層のスタックで構成されています。
モデルのこの部分は事前訓練ですでに十分に訓練されており，[論文](https://arxiv.org/pdf/2006.13979.pdf) で述べられているように，これ以上の微調整は必要ありません。
そのため，*特徴抽出* 部分のすべてのパラメータについて，`requires_grad` を `False` に設定することができます。
<!-- The first component of most transformer-based speech models consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. 
This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part. -->

最後のステップでは，訓練に関連するすべてのパラメータを定義します。
いくつかのパラメータについて詳しく説明します。
<!-- In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:-->

- `group_by_length` は，入力の長さが似ている学習サンプルを 1 つのバッチにまとめることで，学習の効率化を図ります。
これにより，モデルに渡される無駄なパディングトークンの数を大幅に減らすことができ，学習時間を大幅に短縮することができます。

<!-- - `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model -->

- `learning_rate` と `weight_decay` は ，微調整が安定するまで，ヒューリスティックにチューニングされました。
なお，これらのパラメータは Timit データセットに強く依存しており，他の音声データセットでは最適ではない可能性があります。

<!--
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.-->

他のパラメータについての詳しい説明は [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments) を参照してください。
<!-- For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments). -->

学習中は 400 ステップごとにチェックポイントがハブに非同期でアップロードされます。
これにより，モデルの訓練中でも，デモウィジェットを使って遊ぶことができます。
<!-- During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. 
It allows you to also play around with the demo widget even while your model is still training. -->

**注意** もし，モデルのチェックポイントをハブにアップロードしたくない場合は，単純に `push_to_hub=False` を設定してください。
<!-- 
**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`. -->

In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

これで，すべての実体 が Trainer に渡され，訓練を開始する準備が整いました。
<!-- Now, all instances can be passed to Trainer and we are ready to start training! -->

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)



---

${}^1$ CTC では，モデルが話者の割合に依存しないようにするために，連続した同一のトークンは単純に 1 つのトークンとしてグループ化されます。
しかし，符号化されたラベルは，モデルの予測トークンに対応していないため，デコード時にグループ化されるべきではありません。
これが、`group_tokens=False`パラメータを渡さなければならない理由です。
このパラメータを渡さなければ `"hello”` のような単語は誤ってエンコードされ `"helo”` としてデコードされてしまいます。
<!-- 
${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. 
However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. 
If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.-->

${}^2$ 空白トークンは，2 つの l の間に空白トークンを強制的に挿入することにより，モデルに `"hello”` のような単語を予測させます。
我々のモデルによる `"hello”` の CTC 適合予測は `[PAD] [PAD] "h" "e" "e" "l" [PAD] "l" "o" "o" [PAD]` となります。
<!-- ${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. 
A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`. -->

### 訓練 <!--### Training-->

訓練は，このノートブックに割り当てられた GPU に応じて 2, 3 時間かかります。
<!-- Training will take a couple of hours depending on the GPU allocated to this notebook. -->

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 3978
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like t

Step,Training Loss,Validation Loss


最終的な WER は 0.3 程度になるはずです。
これは，最先端の音素誤り率 (PER) が 0.1 をわずかに下回ること（[leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit) 参照) や，WER は通常 PER よりも悪いことを考えると妥当な値です。
<!-- The final WER should be around 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.-->

訓練結果をハブにアップロードするには，以下の命令を実行します。
<!-- 
You can now upload the result of the training to the Hub, just execute this instruction: -->

In [ ]:
trainer.push_to_hub()

このモデルを友人や家族，お気に入りのペットと共有することができます。
例えば 「あなたのユーザ名/あなたが選んだ名前」という識別子でロードすることができます。
<!-- You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance: -->

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
```

CTC 損失を使ってより大きなデータセットでより大きなモデルを微調整するには，公式の音声認識の例 [こちら](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) を見てみるとよいでしょう🤗。
<!-- To fine-tune larger models on larger datasets using CTC loss, one should take a look at the official speech-recognition examples [here](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 🤗. -->